<a href="https://colab.research.google.com/github/Gojo-Satoru-git/GEN-AI/blob/main/Resume%20Analyser(without%20Langchain).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip uninstall numpy
!pip install numpy

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.0.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-99b71e71.so
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 80.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is i

In [4]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


In [7]:
import json
import re
import os
import torch
import docx2txt
import PyPDF2
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

# Load NLP model from Hugging Face (BERT for Named Entity Recognition)
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER")
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def extract_text_from_resume(file_path):
    ext = os.path.splitext(file_path)[-1].lower()
    if ext == ".pdf":
        text = ""
        with open(file_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                text += page.extract_text() + " "
    elif ext in [".doc", ".docx"]:
        text = docx2txt.process(file_path)
    else:
        raise ValueError("Unsupported file format")
    return text.strip()
def extract_resume_details(text):
    extracted_info = {
        "Name": "",
        "Contact": [],
        "Skills": [],
        "Education": "",
        "Experience": "",
        "Certifications": ""
    }

    # Extract Name
    ner_results = ner_pipeline(text)
    for entity in ner_results:
        if entity['entity'] == "B-PER":
            extracted_info["Name"] += entity['word'] + " "

    # Extract Contact Information
    emails = re.findall(r'[\w\.-]+@[\w\.-]+', text)
    phones = re.findall(r'\+?\d[\d\s\-()]{8,}\d', text)
    extracted_info["Contact"] = emails + phones

    # Extract Skills
    skill_keywords = ["Cloth Designing", "Textile Selection", "Industrial Fashion Trends",
                      "Pattern Making", "Sewing", "Garment Construction", "Adobe Illustrator",
                      "Photoshop"]
    extracted_info["Skills"] = [word for word in skill_keywords if word.lower() in text.lower()]

    # Extract Education
    education_match = re.search(r"(Bachelor|Master|PhD) of [A-Za-z ]+", text)
    if education_match:
        extracted_info["Education"] = education_match.group()

    # Extract Experience
    experience_match = re.search(r"(\d{1,2}\+?) years? of experience", text)
    if experience_match:
        extracted_info["Experience"] = experience_match.group()

    # Extract Certifications
    cert_match = re.search(r"Certified [A-Za-z ]+", text)
    if cert_match:
        extracted_info["Certifications"] = cert_match.group()

    return extracted_info


def compare_skills(extracted_skills, job_description):
    job_embedding = embedding_model.encode(job_description, convert_to_tensor=True)
    skill_embeddings = [embedding_model.encode(skill, convert_to_tensor=True) for skill in extracted_skills]

    matching_skills = []
    for skill, emb in zip(extracted_skills, skill_embeddings):
        similarity = util.pytorch_cos_sim(job_embedding, emb).item()
        if similarity > 0.5:  # Threshold for relevance
            matching_skills.append(skill)

    score = (len(matching_skills) / len(extracted_skills)) * 100 if extracted_skills else 0
    return matching_skills, round(score, 2)

def analyze_resume(file_path, job_description):
    text = extract_text_from_resume(file_path)
    extracted_info = extract_resume_details(text)

    matching_skills, fit_score = compare_skills(extracted_info.get("Skills", []), job_description)
    extracted_info["Matching Skills"] = matching_skills
    extracted_info["Fit Score"] = fit_score

    return json.dumps(extracted_info, indent=4)

if __name__ == "__main__":
    file_path = "Resume_Sample.pdf"  # Change to your actual file path
    job_desc = "We are looking for a Cloth designers with industrial experiance"
    result = analyze_resume(file_path, job_desc)
    print(result)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


{
    "Name": "John ",
    "Contact": [
        "johndoe@example.com",
        "+1 234 567 8901"
    ],
    "Skills": [
        "Cloth Designing",
        "Textile Selection",
        "Industrial Fashion Trends",
        "Pattern Making",
        "Sewing",
        "Garment Construction",
        "Adobe Illustrator",
        "Photoshop"
    ],
    "Education": "Bachelor of Arts in Fashion Design",
    "Experience": "",
    "Certifications": "Certified Fashion Designer ",
    "Matching Skills": [
        "Cloth Designing",
        "Textile Selection",
        "Industrial Fashion Trends",
        "Garment Construction"
    ],
    "Fit Score": 50.0
}
